In [1]:
# Author: Zhengxiang (Jack) Wang 
# Date: 2022-01-16, modified on 2022-01-17
# GitHub: https://github.com/jaaack-wang 

## Get paddle

In case you have not installed paddle,run the following cell.

In [2]:
!python3 -m pip install paddlepaddle
#pip3 install paddlepaddle

## Get Data

In case you have not run the `1 - get_data.ipynb`, run the following cell.

In [3]:
!wget http://qim.fs.quoracdn.net/quora_duplicate_questions.tsv
import get_data

get_data.get_quora_data()

--2022-01-17 21:35:43--  http://qim.fs.quoracdn.net/quora_duplicate_questions.tsv
Resolving qim.fs.quoracdn.net (qim.fs.quoracdn.net)... 151.101.53.2
Connecting to qim.fs.quoracdn.net (qim.fs.quoracdn.net)|151.101.53.2|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 58176133 (55M) [text/tab-separated-values]
Saving to: ‘quora_duplicate_questions.tsv.2’

quora_duplicate_que 100%[===================>]  55.48M  13.6MB/s    in 4.1s    

2022-01-17 21:35:48 (13.5 MB/s) - ‘quora_duplicate_questions.tsv.2’ saved [58176133/58176133]

train.txt has been saved!
dev.txt has been saved!
test.txt has been saved!


## Preprocess and numericalize text data

In case you have not run the `2 - preprocess_data.ipynb`, run the following cell.

In [4]:
from utils import *

# ---- load dataset ----
train_set, dev_set, test_set = load_dataset(['train.txt', 'dev.txt', 'test.txt'])

# ---- numericalize the train set ----
V = TextVectorizer(tokenize) 
text = gather_text(train_set) # for collecting texts from train set
V.build_vocab(text) # for building mapping vocab_to_idx dictionary and text_encoder
train_set_encoded = list(encode_dataset(train_set, encoder=V.text_encoder)) # encodoing train set
dev_set_encoded = list(encode_dataset(dev_set, encoder=V.text_encoder)) # encodoing dev set for validation
test_set_encoded  = list(encode_dataset(test_set, encoder=V.text_encoder)) # encodoing dev set for prediction

# ---- build mini batches for the train and dev set ----
train_set_batched = build_batches(train_set_encoded, batch_size=64, include_seq_len=False)
dev_set_batched = build_batches(dev_set_encoded, batch_size=64, include_seq_len=False)
test_set_batched = build_batches(test_set_encoded, batch_size=64, include_seq_len=False)

Two vocabulary dictionaries have been built!
Please call X.vocab_to_idx | X.idx_to_vocab to find out more where [X] stands for the name you used for this TextVectorizer class.


## Training and evaluating models 

### BoW (Bag of Words) model

#### Training

In [5]:
from paddle_models.BoW import BoW
import paddle

In [6]:
def get_model(model):
    model = paddle.Model(model)
    optimizer = paddle.optimizer.Adam(
    parameters=model.parameters(), learning_rate=5e-4)
    criterion = paddle.nn.CrossEntropyLoss()
    metric = paddle.metric.Accuracy()
    model.prepare(optimizer, criterion, metric)
    return model

In [7]:
model = BoW(len(V.vocab_to_idx), 2)
model = get_model(model)
%time model.fit(train_set_batched, dev_set_batched, epochs=5, save_dir='./ckpt', verbose=1)

The loss value printed in the log is the current step, and the metric is the average value of previous steps.
Epoch 1/5
step 20/47 [===========>..................] - loss: 0.6829 - acc: 0.5273 - ETA: 0s - 7ms/step

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/paddle/fluid/layers/utils.py:77: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.10 it will stop working
  return (isinstance(seq, collections.Sequence) and


step 47/47 [==============================] - loss: 0.6580 - acc: 0.5800 - 7ms/step          
save checkpoint at /Users/wzx/Documents/GitHub/text_matching/ckpt/0
Eval begin...
step 16/16 [==============================] - loss: 0.6542 - acc: 0.6500 - 2ms/step          
Eval samples: 1000
Epoch 2/5
step 47/47 [==============================] - loss: 0.4444 - acc: 0.7923 - 6ms/step          
save checkpoint at /Users/wzx/Documents/GitHub/text_matching/ckpt/1
Eval begin...
step 16/16 [==============================] - loss: 0.5743 - acc: 0.6830 - 2ms/step          
Eval samples: 1000
Epoch 3/5
step 47/47 [==============================] - loss: 0.2149 - acc: 0.9057 - 7ms/step          
save checkpoint at /Users/wzx/Documents/GitHub/text_matching/ckpt/2
Eval begin...
step 16/16 [==============================] - loss: 0.6446 - acc: 0.6610 - 2ms/step          
Eval samples: 1000
Epoch 4/5
step 47/47 [==============================] - loss: 0.1095 - acc: 0.9593 - 6ms/step          
save chec

#### Evaluation on the test set

In [8]:
model.evaluate(test_set_batched)

Eval begin...
step 10/16 - loss: 0.8390 - acc: 0.6359 - 2ms/step
step 16/16 - loss: 0.8447 - acc: 0.6330 - 2ms/step
Eval samples: 1000


{'loss': [0.8446506], 'acc': 0.633}

### CNN (Convolutional Neural Network) model

#### Training

In [9]:
from paddle_models.CNN import CNN

model = CNN(len(V.vocab_to_idx), 2)
model = get_model(model)
%time model.fit(train_set_batched, dev_set_batched, epochs=5, save_dir='./ckpt', verbose=1)

The loss value printed in the log is the current step, and the metric is the average value of previous steps.
Epoch 1/5
step 47/47 [==============================] - loss: 0.6740 - acc: 0.5570 - 68ms/step          
save checkpoint at /Users/wzx/Documents/GitHub/text_matching/ckpt/0
Eval begin...
step 16/16 [==============================] - loss: 0.6836 - acc: 0.6270 - 28ms/step          
Eval samples: 1000
Epoch 2/5
step 47/47 [==============================] - loss: 0.5149 - acc: 0.7710 - 71ms/step          
save checkpoint at /Users/wzx/Documents/GitHub/text_matching/ckpt/1
Eval begin...
step 16/16 [==============================] - loss: 0.6256 - acc: 0.6500 - 25ms/step          
Eval samples: 1000
Epoch 3/5
step 47/47 [==============================] - loss: 0.2376 - acc: 0.8853 - 73ms/step          
save checkpoint at /Users/wzx/Documents/GitHub/text_matching/ckpt/2
Eval begin...
step 16/16 [==============================] - loss: 0.6102 - acc: 0.6670 - 25ms/step          
Eval s

#### Evaluation on the test set

In [10]:
model.evaluate(test_set_batched)

Eval begin...
step 10/16 - loss: 0.9297 - acc: 0.6578 - 24ms/step
step 16/16 - loss: 0.7140 - acc: 0.6520 - 25ms/step
Eval samples: 1000


{'loss': [0.7140364], 'acc': 0.652}

## RNN (Recurrent neural network) models

As the RNN models also take as an input the sequence length, we need to re-encode the train set, dev set, and test set. 

In [11]:
# ---- build mini batches for the train and dev set ----
train_set_batched = build_batches(train_set_encoded, batch_size=64, include_seq_len=True)
dev_set_batched = build_batches(dev_set_encoded, batch_size=64, include_seq_len=True)
test_set_batched = build_batches(test_set_encoded, batch_size=64, include_seq_len=True)

### Simple RNN model

#### Training

In [12]:
from paddle_models.S_RNN import SimpleRNN

model = SimpleRNN(len(V.vocab_to_idx), 2, bidirectional=False)
model = get_model(model)
%time model.fit(train_set_batched, dev_set_batched, epochs=5, save_dir='./ckpt', verbose=1)

The loss value printed in the log is the current step, and the metric is the average value of previous steps.
Epoch 1/5
step 47/47 [==============================] - loss: 0.6891 - acc: 0.5133 - 20ms/step          
save checkpoint at /Users/wzx/Documents/GitHub/text_matching/ckpt/0
Eval begin...
step 16/16 [==============================] - loss: 0.6867 - acc: 0.5070 - 7ms/step          
Eval samples: 1000
Epoch 2/5
step 47/47 [==============================] - loss: 0.5232 - acc: 0.7067 - 20ms/step          
save checkpoint at /Users/wzx/Documents/GitHub/text_matching/ckpt/1
Eval begin...
step 16/16 [==============================] - loss: 0.6104 - acc: 0.6050 - 8ms/step          
Eval samples: 1000
Epoch 3/5
step 47/47 [==============================] - loss: 0.1380 - acc: 0.8770 - 22ms/step          
save checkpoint at /Users/wzx/Documents/GitHub/text_matching/ckpt/2
Eval begin...
step 16/16 [==============================] - loss: 0.7188 - acc: 0.6040 - 8ms/step          
Eval samp

#### Evaluation on the test set

In [13]:
model.evaluate(test_set_batched)

Eval begin...
step 10/16 - loss: 1.4090 - acc: 0.5891 - 7ms/step
step 16/16 - loss: 1.3625 - acc: 0.5670 - 7ms/step
Eval samples: 1000


{'loss': [1.3625097], 'acc': 0.567}

### GRU (Gated recurrent units) model 

#### Training

In [14]:
from paddle_models.GRU import GRU

model = GRU(len(V.vocab_to_idx), 2, bidirectional=False)
model = get_model(model)
%time model.fit(train_set_batched, dev_set_batched, epochs=5, save_dir='./ckpt', verbose=1)

The loss value printed in the log is the current step, and the metric is the average value of previous steps.
Epoch 1/5
step 47/47 [==============================] - loss: 0.6846 - acc: 0.5267 - 46ms/step          
save checkpoint at /Users/wzx/Documents/GitHub/text_matching/ckpt/0
Eval begin...
step 16/16 [==============================] - loss: 0.6777 - acc: 0.5350 - 19ms/step          
Eval samples: 1000
Epoch 2/5
step 47/47 [==============================] - loss: 0.4652 - acc: 0.7327 - 49ms/step          
save checkpoint at /Users/wzx/Documents/GitHub/text_matching/ckpt/1
Eval begin...
step 16/16 [==============================] - loss: 0.4449 - acc: 0.6560 - 20ms/step          
Eval samples: 1000
Epoch 3/5
step 47/47 [==============================] - loss: 0.1326 - acc: 0.8673 - 47ms/step          
save checkpoint at /Users/wzx/Documents/GitHub/text_matching/ckpt/2
Eval begin...
step 16/16 [==============================] - loss: 0.6085 - acc: 0.6340 - 20ms/step          
Eval s

#### Evaluation on the test set

In [15]:
model.evaluate(test_set_batched)

Eval begin...
step 10/16 - loss: 1.0250 - acc: 0.6219 - 19ms/step
step 16/16 - loss: 2.1369 - acc: 0.6340 - 18ms/step
Eval samples: 1000


{'loss': [2.1368642], 'acc': 0.634}

### LSTM (Long short-term memory) model

#### Training

In [16]:
from paddle_models.LSTM import LSTM

model = LSTM(len(V.vocab_to_idx), 2, bidirectional=False)
model = get_model(model)
%time model.fit(train_set_batched, dev_set_batched, epochs=5, save_dir='./ckpt', verbose=1)

The loss value printed in the log is the current step, and the metric is the average value of previous steps.
Epoch 1/5
step 47/47 [==============================] - loss: 0.6887 - acc: 0.5163 - 57ms/step          
save checkpoint at /Users/wzx/Documents/GitHub/text_matching/ckpt/0
Eval begin...
step 16/16 [==============================] - loss: 0.6854 - acc: 0.5000 - 21ms/step          
Eval samples: 1000
Epoch 2/5
step 47/47 [==============================] - loss: 0.5163 - acc: 0.7007 - 59ms/step          
save checkpoint at /Users/wzx/Documents/GitHub/text_matching/ckpt/1
Eval begin...
step 16/16 [==============================] - loss: 0.5120 - acc: 0.6640 - 22ms/step          
Eval samples: 1000
Epoch 3/5
step 47/47 [==============================] - loss: 0.1846 - acc: 0.8377 - 58ms/step          
save checkpoint at /Users/wzx/Documents/GitHub/text_matching/ckpt/2
Eval begin...
step 16/16 [==============================] - loss: 0.4665 - acc: 0.6510 - 21ms/step          
Eval s

#### Evaluation on the test set

In [17]:
model.evaluate(test_set_batched)

Eval begin...
step 10/16 - loss: 1.1004 - acc: 0.6234 - 22ms/step
step 16/16 - loss: 1.8570 - acc: 0.6380 - 21ms/step
Eval samples: 1000


{'loss': [1.8569803], 'acc': 0.638}